# Multi-Agent Research Assistant with CrewAI

This notebook demonstrates building a **multi-agent research assistant** using CrewAI. Two agents collaborate sequentially:
1. A **Market Researcher** gathers insights from the web using the Serper search tool
2. A **Product Strategist** creates a positioning strategy based on those insights

## What You'll Learn

- How to build a **multi-agent system** where agents have distinct roles
- How to equip agents with **tools** (e.g., `SerperDevTool` for web search)
- How tasks flow from one agent to the next in a **sequential pipeline**
- How CrewAI's `planning=True` mode orchestrates the overall workflow

## Prerequisites

- `OPENAI_API_KEY` set in your environment
- `SERPER_API_KEY` set in your environment (get one free at [serper.dev](https://serper.dev))

## Step 1: Import Dependencies

We import CrewAI's core classes along with `SerperDevTool` from `crewai_tools`, which gives agents the ability to search the web via the Serper API.

In [1]:
from crewai import Agent, Task, Crew, LLM
import os
from crewai_tools import SerperDevTool  # Web search tool powered by serper.dev
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
llm = LLM(
    model="openai/gpt-oss-20b",
    provider="openai",
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"],
    temperature=0,
)

## Step 2: Configure the Research Target

Define the product to research and any additional context for the strategist agent. These variables are injected into task descriptions via f-strings.

In [3]:
product_name = "energy drink"                     # The product to research and strategize for
strategist_backstory = "and marketing strategy"    # Additional expertise area for the strategist

## Step 3: Define the Agents

We create two specialized agents:

| Agent | Role | Superpower |
|-------|------|-----------|
| **Market Researcher** | Gathers real-time market data | Has access to `SerperDevTool` for web search |
| **Product Strategist** | Turns insights into strategy | Relies on the researcher's output (no tools needed) |

Notice that only the researcher has tools — the strategist works purely from the context it receives.

In [4]:
# Agent 1: Researcher — equipped with web search capability
market_researcher = Agent(
    llm=llm,
    role="Market Researcher",
    goal="Analyze market trends for the product launch",
    backstory="Experienced in market trends and consumer behavior analysis",
    tools=[SerperDevTool()],  # Gives the agent live web search ability
    verbose=True,
)

# Agent 2: Strategist — synthesizes research into actionable strategy
strategist = Agent(
    llm=llm,
    role="Product Strategist",
    goal="Create effective positioning strategies for the product",
    backstory=f"Skilled in competitive positioning {strategist_backstory}",
    verbose=True,
)

## Step 4: Define the Tasks

Tasks define the work each agent performs. The second task implicitly depends on the first because CrewAI executes tasks sequentially by default — the strategist will receive the researcher's output as context.

In [5]:
# Task 1: Researcher searches the web for market intelligence
gather_market_insights_task = Task(
    description=(
        f"Browse the internet to gather insights on current market trends "
        f"for the launch of the {product_name} product."
    ),
    expected_output=f"List of relevant market trends and consumer preferences, relevant to {product_name}",
    agent=market_researcher,
)

# Task 2: Strategist builds on the researcher's findings
develop_positioning_strategy_task = Task(
    description=(
        f"Based on the market insights, create a positioning strategy for "
        f"the {product_name} product, including analysis for impact and target audience."
    ),
    expected_output="A positioning strategy with target audience and impact notes",
    agent=strategist,
)

## Step 5: Assemble the Crew & Execute

The Crew connects agents to tasks. With `planning=True`, CrewAI will first generate an execution plan, then run each task in order. The researcher searches the web first, then the strategist uses those findings to craft a positioning strategy.

In [6]:
crew = Crew(
    agents=[market_researcher, strategist],
    tasks=[gather_market_insights_task, develop_positioning_strategy_task],
    planning=True,  # Auto-generates an execution plan before running
    planning_llm=llm,  # Use our Groq LLM for planning too (defaults to OpenAI otherwise)
)

result = crew.kickoff()
print(result)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Market Researcher                                                                                       │
│                                                                                                                 │
│  Task: Browse the internet to gather insights on current market trends for the launch of the energy drink       │
│  product.1. Define a set of precise search queries that cover all relevant aspects of the energy drink market:  │
│     - "energy drink market trends 2024"                                                                         │
│     - "consumer preferences energy drinks"                                                                      │
│     - "energy drink launch success factors"                                                                     │
│     - "energy drink regulatory trends"                                                                          │
│     - "energy drink competitor analysis"                                                                        │
│                                                                                                                 │
│  2. For each query, use the SerperDevTool to perform a web search:                                              │
│     - Call the tool with the search_query parameter set to the current query.                                   │
│     - Retrieve the top 10 results for each query.                                                               │
│                                                                                                                 │
│  3. Extract key insights from each result:                                                                      │
│     - Identify the main trend highlighted.                                                                      │
│     - Note any consumer preference data (e.g., flavor, packaging, price sensitivity).                           │
│     - Record market size, growth rate, and forecast figures.                                                    │
│     - Capture any regulatory or compliance information relevant to product launch.                              │
│     - Summarize competitor positioning and market share.                                                        │
│                                                                                                                 │
│  4. Organize the extracted data into a structured format:                                                       │
│     - Create categories: Trend, Consumer Preference, Market Size & Growth, Regulatory, Competitor.              │
│     - For each category, list the most relevant findings with source URLs.                                      │
│                                                                                                                 │
│  5. Compile the organized insights into a single document or spreadsheet.                                       │
│                                                                                                                 │
│  6. Deliver the final list of relevant market trends and consumer preferences, ensuring it aligns with the      │
│  task_expected_output specification.                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Tool search_the_internet_with_serper executed with result: {'searchParameters': {'q': 'energy drink market trends 2024', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': '2024 Energy drink trends - Glanbia', 'link': 'https://www.glanbia...
Tool search_the_internet_with_serper executed with result: {'searchParameters': {'q': 'consumer preferences energy drinks', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': "What's the Buzz? Preferences and perceptions of policies to re...
Tool search_the_internet_with_serper executed with result: {'searchParameters': {'q': 'energy drink launch success factors', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'How Innovation is Redefining the Energy Drink Market? | BevNE...
Tool search_the_internet_with_serper executed with result: {'searchParameters': {'q': 'energy drink regulatory trends', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Patterns of Energy

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Market Researcher                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Energy‑Drink Market Insights – 2024 / 2025**                                                                 │
│                                                                                                                 │
│  | Category | Key Findings | Source |                                                                           │
│  |----------|--------------|--------|                                                                           │
│  | **Trend** | • Amino‑acid‑infused “sports‑nutrition” variants are gaining traction.<br>• Fruit‑flavor         │
│  expansion (e.g., tropical, berry) dominates new SKUs.<br>• Low‑/no‑sugar claims and natural ingredient         │
│  positioning are rising.<br>• Hybrid coffee/tea and seltzer‑based energy drinks are the fastest‑growing         │
│  sub‑segments.<br>• Functional claims (cognitive boost, energy without jitters) are becoming core               │
│  differentiators. | 1. Glanbia “Energy Drink Trends 2024” –                                                     │
│  https://www.glanbianutritionals.com/en/nutri-knowledge-center/insights/energy-drink-trends-what-expect<br>2.   │
│  NIQ “Navigating the Dynamic Energy Drink Market” –                                                             │
│  https://nielseniq.com/global/en/insights/education/2024/energy-drink-trends/ |                                 │
│  | **Consumer Preference** | • Taste is the top priority (≈35 % of respondents).<br>• Caffeine content is       │
│  equally important (≈35 %).<br>• Flavor preferences: fruit (especially tropical/berry), coffee‑tea hybrids,     │
│  and “refreshing” profiles.<br>• Low‑sugar or no‑sugar options are sought by ~30 % of consumers.<br>•           │
│  Convenient single‑serve packaging and “on‑the‑go” formats are preferred.<br>• Price sensitivity is moderate;   │
│  consumers are willing to pay a premium for functional claims. | 1. Glanbia “What Consumers Look for in Energy  │
│  Drinks” –                                                                                                      │
│  https://www.glanbianutritionals.com/en/nutri-knowledge-center/insights/energy-drink-trends-what-expect<br>2.   │
│  Mintel “US Energy Drinks Market Analysis & Forecast 2025‑2030” –                                               │
│  https://store.mintel.com/report/us-energy-drinks-market-report |                                               │
│  | **Market Size & Growth** | • US market ≈ $23.9 B in 2024; CAGR 8.3 % to 2033 (US).<br>• Global market ≈      │
│  $79.4 B in 2024; projected to reach $125.1 B by 2030 (CAGR ≈ 8 %).<br>• North America remains the largest      │
│  region; Asia‑Pacific shows fastest growth (≈ 9 % CAGR).<br>• Projected global CAGR 2024‑2034 ≈ 5.4 %. | 1.     │
│  Mintel “US Energy Drinks Market Analysis & Forecast 2025‑2030” –                                               │
│  https://store.mintel.com/report/us-energy-drinks-market-report<br>2. Grand View Research “Energy Drinks        │
│  Market Size & Share” – https://www.grandviewresearch.com/industry-analysis/energy-drinks-market |              │
│  | **Regulatory** | • Poland: ban on sale of caffeine‑taurine energy drinks to minors (under 18) effective 1    │
│  Jan 2024.<br>• US FDA treats energy drinks as dietary supplements; no pre‑market approval required, but        │
│  ingredients must be GRAS or have safety data.<br>• EU:

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Product Strategist                                                                                      │
│                                                                                                                 │
│  Task: Based on the market insights, create a positioning strategy for the energy drink product, including      │
│  analysis for impact and target audience.1. Review the compiled market insights from Task 1.                    │
│                                                                                                                 │
│  2. Identify gaps and opportunities:                                                                            │
│     - Highlight unmet consumer needs.                                                                           │
│     - Spot emerging trends that the product can leverage.                                                       │
│     - Note regulatory constraints that could shape positioning.                                                 │
│                                                                                                                 │
│  3. Craft a clear positioning statement:                                                                        │
│     - Define the unique value proposition.                                                                      │
│     - Articulate the core benefit the product delivers.                                                         │
│     - Ensure the statement is concise and memorable.                                                            │
│                                                                                                                 │
│  4. Define the target audience:                                                                                 │
│     - Segment by demographics (age, gender, income).                                                            │
│     - Segment by psychographics (lifestyle, values, motivations).                                               │
│     - Segment by behavior (purchase frequency, brand loyalty, channel preference).                              │
│                                                                                                                 │
│  5. Analyze impact:                                                                                             │
│     - Estimate potential market share based on identified opportunities.                                        │
│     - Project revenue impact using market size and penetration assumptions.                                     │
│     - Assess brand equity benefits (e.g., differentiation, premium positioning).                                │
│                                                                                                                 │
│  6. Document the positioning strategy:                                                                          │
│     - Include the positioning statement.                                                                        │
│     - List the target audience segments.                                                                        │
│     - Provide impact notes with quantitative and qualitative insights.                                          │
│                                                                                                                 │
│  7. Deliver the final positioning strategy document, ensuring it meets the task_expected_output requirements.   │
│                                                                                                                 │
╰────────────────────────────────────────────────────────

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Product Strategist                                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Positioning Strategy for “VitaBoost” – A Next‑Generation Energy Drink**                                      │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ## 1. Market Insight Snapshot (2024‑2025)                                                                      │
│                                                                                                                 │
│  | Insight | Detail |                                                                                           │
│  |---------|--------|                                                                                           │
│  | **Trend Drivers** | • Amino‑acid‑infused sports‑nutrition variants<br>• Fruit‑flavor expansion               │
│  (tropical/berry)<br>• Low‑/no‑sugar & natural‑ingredient positioning<br>• Hybrid coffee/tea & seltzer‑based    │
│  formats<br>• Cognitive‑boost & “no‑jitters” functional claims |                                                │
│  | **Consumer Priorities** | • Taste (35 %) & caffeine (35 %) top drivers<br>• 30 % seek low‑sugar              │
│  options<br>• Single‑serve, on‑the‑go packaging preferred |                                                     │
│  | **Market Size** | • US: $23.9 B (2024) – CAGR 8.3 % to 2033<br>• Global: $79.4 B (2024) – CAGR 8 % to 2030   │
│  |                                                                                                              │
│  | **Regulatory Landscape** | • Age‑restriction labeling (e.g., Poland, EU)<br>• FDA treats as dietary          │
│  supplement – ingredients must be GRAS<br>• EU tightening on caffeine limits & health‑claim substantiation |    │
│  | **Competitive Landscape** | • Red Bull (30 %) & Monster (25 %) dominate<br>• Emerging players: Celsius       │
│  (clean‑energy), Bang (high‑caffeine), G Fuel (gaming)<br>• Success drivers: distribution, functional           │
│  differentiation, price‑positioning |                                                                           │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ## 2. Gap & Opportunity Analysis                                                                               │
│                                                                                                                 │
│  | Gap / Opportunity | Why It Matters | How VitaBoost Leverages It |                                            │
│  |-------------------|----------------|---------------------------|                                             │
│  | **Unmet Need: “Clean‑Energy + Cognitive Boost”** | Consumers want energy without the “jitters” and a clear   │
│  mental edge. | Amino‑acid blend (L‑tyrosine, L‑carnitine) + natural caffeine (green tea extract) + no‑sugar    │
│  sweetener. |                                          

**Positioning Strategy for “VitaBoost” – A Next‑Generation Energy Drink**

---

## 1. Market Insight Snapshot (2024‑2025)

| Insight | Detail |
|---------|--------|
| **Trend Drivers** | • Amino‑acid‑infused sports‑nutrition variants<br>• Fruit‑flavor expansion (tropical/berry)<br>• Low‑/no‑sugar & natural‑ingredient positioning<br>• Hybrid coffee/tea & seltzer‑based formats<br>• Cognitive‑boost & “no‑jitters” functional claims |
| **Consumer Priorities** | • Taste (35 %) & caffeine (35 %) top drivers<br>• 30 % seek low‑sugar options<br>• Single‑serve, on‑the‑go packaging preferred |
| **Market Size** | • US: $23.9 B (2024) – CAGR 8.3 % to 2033<br>• Global: $79.4 B (2024) – CAGR 8 % to 2030 |
| **Regulatory Landscape** | • Age‑restriction labeling (e.g., Poland, EU)<br>• FDA treats as dietary supplement – ingredients must be GRAS<br>• EU tightening on caffeine limits & health‑claim substantiation |
| **Competitive Landscape** | • Red Bull (30 %) & Monster (25 %) dominate<br>• Emerging 